In [1]:
import pandas as pd

In [23]:
data = pd.read_csv('/Users/camille/repo/Hetic/projet_gouv/scraping/Cleaning_eda/all_data_neg.csv')

In [28]:
data

,rate,review_text,object_address,cleaned_text,sentiment,neg_reviews_terms
0,1,Pas de formation possible Pas de financement p...,"5 Rue de Quimper, 68100 Mulhouse",formation financement refus refus parcours inu...,0,"['financement', 'refu', 'refu', 'parcour', 'in..."
1,1,"Aucun respect, j’explique pourtant bien mon so...","5 Rue de Quimper, 68100 Mulhouse",aucun respect pourtant bien souci faire rond p...,0,['souci']
2,1,"Les fonctionnaires incompétents, arrogants et ...","5 Rue de Quimper, 68100 Mulhouse",fonctionnaire incompétent arrogant prétentieux...,0,"['prétentieux', 'bunker', 'pauvre', 'classe', ..."
3,1,"Aucun respect, j’explique pourtant bien mon so...","5 Rue de Quimper, 68100 Mulhouse",aucun respect pourtant bien souci air hautain ...,0,"['respect', 'souci', 'hautain', 'discriminant'..."
4,1,Si je pouvais ne pas mettre d'étoiles je n'en ...,"5 Rue de Quimper, 68100 Mulhouse",étoile mettrai conseiller indemnisation désagr...,0,"['mettrai', 'désagréable', 'agacer', 'rappel',..."
...,...,...,...,...,...,...
5879,1,"La gentillesse chez vous , c’est pas une quali...","45 Av. Billaud Varenne, 17000 La Rochelle",gentillesse qualité désagréable accueil,0,"['qualité', 'désagréable']"
5880,1,Le 3949 est le pire numéro de renseignement qu...,"45 Av. Billaud Varenne, 17000 La Rochelle",3949 pire numéro renseignement malheur,0,['pire']
5881,1,Incompétents.,"45 Av. Billaud Varenne, 17000 La Rochelle",incompétent,0,[]
5882,1,Sourires en option,"45 Av. Billaud Varenne, 17000 La Rochelle",sourire option,0,[]


In [24]:
data.dropna(subset=['review_text'], inplace=True)

In [3]:
data['pos'] = data['rate'].apply(lambda x:  x >= 2.5)
df_pos = data[data['pos'] == True].reset_index(drop=True)
df_neg = data[data['pos'] == False].reset_index(drop=True)

In [4]:
from bertopic import BERTopic



/Users/camille/repo/Hetic/projet_gouv/scraping/gouv_env/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/camille/repo/Hetic/projet_gouv/scraping/gouv_env/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/camille/repo/Hetic/pr

In [25]:
topic_model = BERTopic()
BERTopic(language="multilingual") 

## SentenceTransformer

In [26]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("paraphrase-albert-small-v2")
topic_model = BERTopic(embedding_model=sentence_model, nr_topics=20)


In [29]:
data['review_text'] = data['review_text'].astype(str)
data.dropna(subset=['cleaned_text'], inplace=True)
data.reset_index(drop=True, inplace=True)
topics, probs = topic_model.fit_transform(data['cleaned_text'])


In [30]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,2845,-1_emploi_conseiller_pôle_rien
1,0,2142,0_agence_conseiller_rien_emploi
2,1,109,1_catastrophique_catastrophe_aucun_conseiller
3,2,80,2_candidat_heure_candidature_plaque
4,3,77,3_mauvais_expérience_vrai_vraiment
5,4,66,4_sérieux_midi_ferme_incroyable
6,5,65,5_radiation_conseiller_menace_rendez
7,6,56,6_financement_employeur_emploi_formation
8,7,43,7_jamais_jambe_somme_dossier
9,8,39,8_parking_place_petit_retard


In [7]:
topic_model.reduce_topics(df_neg['review_text'], nr_topics=10)
topics = topic_model.topics_
probabilities = topic_model.probabilities_


ValueError: This BERTopic instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [51]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,2374,-1_de_et_pas_je
1,0,3220,0_de_et_pas_que
2,1,62,1_the_to_and_was
3,2,53,2_papiers_ils_me_et
4,3,50,3_numéro_téléphone_numero_de
5,4,39,4_nul_nuls_importe_grrrrrrrrrrr
6,5,31,5_fermé_fermeture_jeudi_midi
7,6,29,6_parking_pas_place_le
8,7,15,7_zero_zéro_null_réactivité
9,8,11,8_inutile_trop_excellence_quasi


In [35]:
topic_model.get_topic(4)

[('sérieux', 0.25134120513687247),
 ('midi', 0.18901733084306668),
 ('ferme', 0.17470395572823105),
 ('incroyable', 0.16804334639530522),
 ('fermé', 0.15974469959670434),
 ('fermeture', 0.12306755088530451),
 ('blague', 0.09481925673156472),
 ('12h30', 0.07848620980264222),
 ('incompétence', 0.07405854702854557),
 ('bravo', 0.06121642139151164)]

In [39]:
essai = topic_model.get_document_info(data['cleaned_text'])

In [40]:
filtered_df = essai[essai['Representative_document'] == True]


In [41]:
filtered_df

,Document,Topic,Name,Top_n_words,Probability,Representative_document
19,very very useles working there had 9.30am and ...,13,13_the_and_was_they,the - and - was - they - she - you - her - did...,1.000000,True
196,conseiller pôle emploi rendez-vous,0,0_agence_conseiller_rien_emploi,agence - conseiller - rien - emploi - pôle - r...,1.000000,True
242,catastrophique,1,1_catastrophique_catastrophe_aucun_conseiller,catastrophique - catastrophe - aucun - conseil...,1.000000,True
290,honte pôle emploi département bien ailleurs ic...,6,6_financement_employeur_emploi_formation,financement - employeur - emploi - formation -...,0.906691,True
469,promotion précarité aide recrutement candidat ...,2,2_candidat_heure_candidature_plaque,candidat - heure - candidature - plaque - impr...,0.381063,True
...,...,...,...,...,...,...
5582,droit avis lettre radiation menace allocation,5,5_radiation_conseiller_menace_rendez,radiation - conseiller - menace - rendez - let...,1.000000,True
5664,inutile,9,9_allocation_inutile_mois_efficace,allocation - inutile - mois - efficace - part ...,1.000000,True
5668,chômage,15,15_chômage_chômeur_chomeur_chomage,chômage - chômeur - chomeur - chomage - royale...,1.000000,True
5761,promotion précarité aide recrutement candidat ...,2,2_candidat_heure_candidature_plaque,candidat - heure - candidature - plaque - impr...,0.387157,True


In [31]:
topic_model.visualize_topics()


In [34]:
hierarchical_topics = topic_model.hierarchical_topics(data['cleaned_text'])
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)



100%|██████████| 18/18 [00:00<00:00, 373.03it/s]


In [23]:
topic_model.get_document_info(df_neg['review_text']).to_csv('data_nlp/df_neg_bert.csv', index=False)


In [26]:
df_neg_bert = pd.read_csv('data_nlp/df_neg_bert.csv')

In [31]:
# voir la premiere ligne du csv 
df_neg_bert.loc[0, 'Document']

'Pas de formation possible\nPas de financement possible\nrefus sur refus de parcours\nEt rendez vous régulier inutile.\nDire que soit disant on est la pour Aider les chômeurs a être réactif.\nAbsurde !'

In [2]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

data = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))
docs = data["data"]
categories = data["target"]
category_names = data["target_names"]

/Users/camille/repo/Hetic/projet_gouv/scraping/gouv_env/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/camille/repo/Hetic/projet_gouv/scraping/gouv_env/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/camille/repo/Hetic/pr

In [8]:
import pandas as pd

df = pd.DataFrame({"Document": docs, "Category": [category_names[i] for i in categories]})


In [9]:
df

,Document,Category
0,\n\nI am sure some bashers of Pens fans are pr...,rec.sport.hockey
1,My brother is in the market for a high-perform...,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,comp.sys.mac.hardware
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,sci.med
18842,\nNot in isolated ground recepticles (usually ...,sci.electronics
18843,I just installed a DX2-66 CPU in a clone mothe...,comp.sys.ibm.pc.hardware
18844,\nWouldn't this require a hyper-sphere. In 3-...,comp.graphics


In [10]:
topic_model = BERTopic(verbose=True).fit(docs, y=categories)


Batches: 100%|██████████| 589/589 [05:04<00:00,  1.94it/s]
2023-05-31 18:54:48,934 - BERTopic - Transformed documents to Embeddings
2023-05-31 18:55:04,947 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2023-05-31 18:55:06,100 - BERTopic - Clustered reduced embeddings


In [11]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,4063,-1_of_to_the_and
1,0,935,0_space_launch_nasa_orbit
2,1,914,1_window_server_file_motif
3,2,912,2_he_game_year_baseball
4,3,900,3_car_cars_engine_ford
...,...,...,...
200,199,10,199_rs232_12v_rs_serial
201,200,10,200_clinic_doctor_anes_emergency
202,201,10,201_turkey_truelove_leftover_christmas
203,202,10,202_winbench_graphite_vlb_ati


In [12]:
topics, probs = topic_model.fit_transform(docs)


Batches:   1%|▏         | 8/589 [00:08<10:31,  1.09s/it]


KeyboardInterrupt: 

In [13]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)


2023-05-31 19:02:11,868 - BERTopic - Transformed documents to Embeddings
2023-05-31 19:02:17,284 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2023-05-31 19:02:18,401 - BERTopic - Clustered reduced embeddings
